In [1]:
with open("example_md_to_text.txt", "r", encoding="latin-1") as f: # TO DO: check proper encoding for .md files
    markdown_example = f.read()

with open("example.pdf", "rb") as f:
    pdf = f.read()

with open("HF_TOKEN.txt", "r") as f:
    hf_token = f.read()

with open("GROQ_KEY.txt", "r") as f:
    groq_token = f.read()

In [2]:
from huggingface_hub import HfFolder, whoami

HfFolder.save_token(hf_token)
print(whoami()["name"])

c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


alberto-lorente


In [3]:
import torch

device = "cpu"
if torch.cuda.is_available():
    print("Cuda available")
    device = torch.device('cuda')

Cuda available


In [4]:
pdf_path = "example.pdf"

base_prompt="""CONTEXTE
L'image suivante contient une page et un tableau. 

TÂCHE
Décrivez le tableau et le contenu de la page en accordant une attention particulière au contexte qui l'entoure, qu'il s'agisse de budgets, de dates, d'élections, d'agendas, de projets futurs ou de sujets connexes. 

FORMAT DE LA RÉPONSE
Votre réponse doit être aussi détaillée que possible. 
Votre résultat doit être la description du tableau directement.
La langue de votre réponse est le français"""


summary_prompt = """Il s'agit d'un texte concernant un projet de géothermie :
TEXTE: 
{}

TÂCHE:
Make a one page summary paying special attention to all the administrative matters like budgets, plans, actions to take in the future, organizational and hierarchical charts, announcements, meetings, contacts, elections, reports and all the related topics to these.

OUTPUT:
Publier directement le résumé.
"""


In [5]:
import gc
def unload_cuda():
    torch.cuda.empty_cache()
    gc.collect()
unload_cuda()

In [6]:
from council_rag.preprocessing import preprocess_markdown_text
from council_rag.data_transformations import process_tables, summarize_clusters
import time

start = time.time()

paragraphs_list, clusters_dict, model = preprocess_markdown_text(markdown_example,
                                                        model_id ="Jaume/gemma-2b-embeddings", 
                                                        spacy_model="fr_core_news_sm", 
                                                        n_sents_per_para=8,
                                                        device=device)

Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  1.51it/s]


In [7]:
unload_cuda()
processed_tables = process_tables(pdf_path, 
                                base_prompt, 
                                groq_token)

In [8]:
clusters_dict.keys()

dict_keys(['cluster_0', 'cluster_1', 'cluster_2', 'cluster_3', 'cluster_4', 'cluster_5', 'cluster_6', 'cluster_7'])

In [9]:
unload_cuda()
clusters_dict = summarize_clusters(clusters_dict, 
                                    summary_prompt, 
                                    groq_token, 
                                    model="gemma2-9b-it", 
                                    token_limit=14000, 
                                    sleep_time=60)

end = time.time()
print((end - start)/60)

15.375812216599781


The variables we are going to query are:

In [10]:
# paragraphs_dict 
# processed_tables
# clusters_dict

## Indexing and querying

Deleting some info that we won't need and turning the cluster dict into a list to match the paragraphs list.

In [11]:
del paragraphs_list[0]["para_embedding"]

In [12]:
clusters_list = list(clusters_dict.values())
i = 0
while i < len(clusters_list):
    clusters_list[i]["cluster"] = i
    i += 1


Getting the embedding model and setting up the index and vector stores.

In [13]:
embedding_model = model

In [14]:
# checking the shape of the embedding to pass it to the index
shape_emb = embedding_model.encode("Hello World!")
emd_dims =  shape_emb.shape[0]

In [15]:
import faiss

# init the intex
index = faiss.IndexFlatL2(emd_dims)

In [16]:
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

# creating the vector store
vector_store = FAISS(embedding_model, 
                    index, 
                    InMemoryDocstore({}), 
                    {})

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [17]:
clusters_list[0].keys()

dict_keys(['para_indexes', 'union_paras', 'cluster_summary', 'cluster'])

Processing all the information we have collected so far into Document objects that will be fed into the vector store.

In [18]:
cluster_paras = [] # to process into smaller chunks
cluster_summ_docs = [] # filled with the cluster summaries langchain doc type

for cluster in clusters_list:
    
    cluster_para = cluster["union_paras"]
    cluster_summ = cluster["cluster_summary"]
    
    cluster_paras.append(cluster_para)
    cluster_summ_docs.append(Document(page_content=cluster_summ, metadata={"cluster": cluster["cluster"],
                                                                            "type": "summary"}))

In [19]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=450, 
                                        chunk_overlap=35,
                                        length_function=len,
                                        is_separator_regex=False)

In [20]:
cluster_paras_docs = []
for cluster_union in cluster_paras:
    cluster_union_docs = splitter.split_text(cluster_union)
    cluster_paras_docs.append(cluster_union_docs)

In [21]:
len(cluster_paras_docs[0])

12

In [22]:
cluster_chunks = []
i = 0
while i < len(cluster_paras_docs):  
    curr_cluster = cluster_paras_docs[i]
    for cluster_chunk in curr_cluster:
        cluster_chunks.append(Document(page_content=cluster_chunk, metadata={"cluster": i,
                                                                                "type": "cluster_chunk"}))
    i += 1


In [23]:
augmented_table_chunks = []
table_descriptions_chunks = []
html_tables_chunks = []

for table in processed_tables:
    augmented_chunk = table["table_augmented_context"]
    augmented_table_chunks.append(Document(page_content=augmented_chunk, metadata={"type": "augmented_table"}))
    
    description_chunk = table["table_context"]
    table_descriptions_chunks.append(Document(page_content=description_chunk, metadata={"type": "description_table"}))
    
    html_chunk = table["table_html"]
    html_tables_chunks.append(Document(page_content=html_chunk, metadata={"type": "html_table"}))
    


Overall we have:

- The individual text chunks.
- The clusters summaries chunks.
- The augmented table chunks.
- The table descriptions chunks.
- The html tables chunks.


In [24]:
cluster_chunks[0]

Document(metadata={'cluster': 0, 'type': 'cluster_chunk'}, page_content="Rue des Vernes à Roanne\n\n\n\n # PROCES VERBAL\n\n\n\n L'an deux mille vingt et un, le 28 octobre à 18 h 00, les conseillers communautaires de Roannais Agglomération, se sont réunis à l\x92Espace Chorum \x96 Halle Vacheresse \x96 Rue des Vernes à Roanne.\n\n\n\nLa convocation de tous les conseillers a été faite le 22 octobre 2021, dans les formes et délais prescrits par la loi, par Yves Nicolin, Président.\n\n\n\n # Etaient présents :")

In [25]:
cluster_summ_docs[0]

Document(metadata={'cluster': 0, 'type': 'summary'}, page_content='## Résumé du Procès-Verbal des Conseillers Communautaires de Roannais Agglomération \n\n**Date:** 28 octobre 2021\n\n**Lieu:** Espace Chorum - Halle Vacheresse - Rue des Vernes à Roanne\n\n**Présidence:** Yves Nicolin\n\nCe procès-verbal résume les décisions prises lors de la réunion du conseil communal de Roannais Agglomération le 28 octobre 2021. \n\n**Permissions et Conventions:**\n\n* **Droit de chasse:** La Fédération Départementale des Chasseurs de la Loire a obtenu un droit de chasse pédagogique sur le site Domaine des Grands Murcins. Ce droit est limité au grand gibier et aux espèces nuisibles, sans agrainage.\n* **Construction Bâtiment d\'Enseignement Supérieur:** Un avenant au contrat du lot 10  "Serrurerie" concernant les travaux de construction d\'un Bâtiment d\'Enseignement Supérieur sur le Campus Mendes France à Roanne (phase 2) a été approuvé avec la société ROCHE.\n* **Avenant Spectacle "BoOm":**  L\'ave

In [26]:
augmented_table_chunks[0]

Document(metadata={'type': 'augmented_table'}, page_content='Cette page et le tableau sont l\'univers de la contestation politique issue du mouvement municipaliste breton. Il est à noter que l\'ensemble des informations, lorsqu\'il les connaissait, ou les prenait pour réelle, se révèle être erroné ou faux.\n\nLa suite de la réponse se trouve dans l\'encadré "SITUATION DE LA PAGE".\n    \nTableau au format html:\n\n<table>\n <tr>\n  <td colspan="1" rowspan="1">\n   Absents\n  </td>\n  <td colspan="1" rowspan="1">\n   Ni pouvoir\n   <br/>\n   Ni suppléant\n  </td>\n  <td colspan="1" rowspan="1">\n   Suppléant\n  </td>\n  <td colspan="1" rowspan="1">\n   Pouvoir donné à…\n  </td>\n </tr>\n <tr>\n  <td colspan="1" rowspan="1">\n   Jean-Marc Ambroise\n  </td>\n  <td colspan="1" rowspan="1">\n   X\n  </td>\n  <td colspan="1" rowspan="1">\n  </td>\n  <td colspan="1" rowspan="1">\n  </td>\n </tr>\n <tr>\n  <td colspan="1" rowspan="1">\n   Laurence Boyer\n  </td>\n  <td colspan="1" rowspan="1">

In [27]:
table_descriptions_chunks[0]

Document(metadata={'type': 'description_table'}, page_content='Cette page et le tableau sont l\'univers de la contestation politique issue du mouvement municipaliste breton. Il est à noter que l\'ensemble des informations, lorsqu\'il les connaissait, ou les prenait pour réelle, se révèle être erroné ou faux.\n\nLa suite de la réponse se trouve dans l\'encadré "SITUATION DE LA PAGE".')

In [28]:
html_tables_chunks[0]

Document(metadata={'type': 'html_table'}, page_content='Cette page et le tableau sont l\'univers de la contestation politique issue du mouvement municipaliste breton. Il est à noter que l\'ensemble des informations, lorsqu\'il les connaissait, ou les prenait pour réelle, se révèle être erroné ou faux.\n\nLa suite de la réponse se trouve dans l\'encadré "SITUATION DE LA PAGE".\n    \nTableau au format html:\n\n<table>\n <tr>\n  <td colspan="1" rowspan="1">\n   Absents\n  </td>\n  <td colspan="1" rowspan="1">\n   Ni pouvoir\n   <br/>\n   Ni suppléant\n  </td>\n  <td colspan="1" rowspan="1">\n   Suppléant\n  </td>\n  <td colspan="1" rowspan="1">\n   Pouvoir donné à…\n  </td>\n </tr>\n <tr>\n  <td colspan="1" rowspan="1">\n   Jean-Marc Ambroise\n  </td>\n  <td colspan="1" rowspan="1">\n   X\n  </td>\n  <td colspan="1" rowspan="1">\n  </td>\n  <td colspan="1" rowspan="1">\n  </td>\n </tr>\n <tr>\n  <td colspan="1" rowspan="1">\n   Laurence Boyer\n  </td>\n  <td colspan="1" rowspan="1">\n  <

In [29]:
all_docs = cluster_chunks + cluster_summ_docs + augmented_table_chunks + table_descriptions_chunks + html_tables_chunks

In [30]:
len(all_docs)

292

In [40]:
ids = [i for i in range(len(all_docs))]
# ids

In [33]:
# checking types of the docs
# for doc in all_docs:
#     print(type(doc))
#     print(doc.metadata)
#     print(doc.page_content)

In [34]:
# help(vector_store.add_documents)

In [49]:
cluster_chunks[0].page_content

"Rue des Vernes à Roanne\n\n\n\n # PROCES VERBAL\n\n\n\n L'an deux mille vingt et un, le 28 octobre à 18 h 00, les conseillers communautaires de Roannais Agglomération, se sont réunis à l\x92Espace Chorum \x96 Halle Vacheresse \x96 Rue des Vernes à Roanne.\n\n\n\nLa convocation de tous les conseillers a été faite le 22 octobre 2021, dans les formes et délais prescrits par la loi, par Yves Nicolin, Président.\n\n\n\n # Etaient présents :"

In [46]:
from council_rag.preprocessing.preprocessing import compute_norm_embeddings

In [ ]:
all_embeddings = [compute_norm_embeddings(embedding_model, doc.page_content) for doc in all_docs]

In [41]:
# adding the documents to the vectorstore

vector_store.add_documents(documents=cluster_chunks, ids=ids)
# if this still does not work, we can do FAISS.from_texts with the embeddings computed manually

AttributeError: 'str' object has no attribute 'items'

UP UNTIL THIS POINT, EVERYTHING IS PACKAGED MISSING THE ACTUAL POPULATING OF THE DB

Querying

In [ ]:
query = "Get from Tunji's thingie"

results = vector_store.similarity_search(
    query,
    k=2,
    filter={"source": "tweet"}, # put the clustering, etc here
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]") # this is how to get the attributes

CHECK WITH THE FIXED VECTOR STORE BEFORE PACKAGING INTO COUNCIL_RAG.RAG AND MAIN.PY

In [7]:
# algo to query the store in the right order
def query_vector_store(vector_store, query):
        # first search through the summaries
        results_query_level_one = vector_store.similarity_search(
                                                                query,
                                                                k=1,
                                                                filter={"type": "summary"}, 
                                                                )

        cluster_level_one = results_query_level_one[0].metadata["cluster"]

        # now we search only in the cluster retrieved in the first step
        results_query_level_two = vector_store.similarity_search(
                                                        query,
                                                        k=5,
                                                        filter={"cluster": cluster_level_one,
                                                                "type": "cluster_chunk"}, 
                                                        )
        results_query_tables = vector_store.similarity_search(
                                                        query,
                                                        k=1,
                                                        filter={"type": 'description_table'}, # {"$in": []} 
                                                        )
        
        relevant_facts = [sent_query_level_two.page_content for sent_query_level_two in results_query_level_two]
        relevant_table = results_query_tables.page_content
        
        return relevant_facts, relevant_table

In [8]:
def augment_query_rag(query, relevant_facts, relevant_table):
    
    aug_prompt="""\nVoici quelques faits pertinents pour vous aider à répondre et una description du qui peut s'avérer utile. 
    Si la description du n'est pas utile, ignorez-le.\n"""
    
    augmented_data_string = aug_prompt + "\n".join(relevant_facts) + "\n" + relevant_table
    
    formated_augmented_query = query.format(augmented_data_string) # the original query has to have a format string
    # formated_augmented_query = query + augmented_data_string       # if that is not the case, we can do this instead
    
    return formated_augmented_query

In [9]:
from groq import Groq

def final_query(query, model, groq_key):
    
    client = Groq(api_key=groq_key)
    
    messages = [
                {"role": "system",
                "content":  "Vous êtes un assistant utile" },
                {"role": "user",
                "content":  query}
                ]
    
    chat_completion = client.chat.completions.create(messages=messages, model=model)
    return chat_completion.choices[0].message.content

In [10]:
# final fx to query our list of queries

def query_multiple(list_of_queries, vector_store, model, groq_key):
    
    responses = []
    for query in list_of_queries:
        
        relevant_facts, relevant_table = query_vector_store(vector_store, query)
        formated_augmented_query = augment_query_rag(query, relevant_facts, relevant_table)
        response = final_query(formated_augmented_query, model, groq_key)
        responses.append(response)
    
    return responses